In [33]:
import pandas as pd
from sklearn.datasets import load_iris
from datetime import datetime
import numpy as np

data = load_iris()
data.feature_names = ['sepal_length_cm',
 'sepal_width_cm',
 'petal_length_cm',
 'petal_width_cm']
df = pd.DataFrame(data= np.c_[data['data'], data['target']],
                     columns=data['feature_names'] + ['target'])
df['event_timestamp'] = [datetime.fromisoformat('2023-02-20') for _ in data.target]
df['iris_id'] = df.index.values
df.head()

,sepal_length_cm,sepal_width_cm,petal_length_cm,petal_width_cm,target,event_timestamp,iris_id
0,5.1,3.5,1.4,0.2,0.0,2023-02-20,0
1,4.9,3.0,1.4,0.2,0.0,2023-02-20,1
2,4.7,3.2,1.3,0.2,0.0,2023-02-20,2
3,4.6,3.1,1.5,0.2,0.0,2023-02-20,3
4,5.0,3.6,1.4,0.2,0.0,2023-02-20,4


In [34]:
!feast init feature_repo


Creating a new Feast repository in /home/sbalawajder/projects/train/feast-mlflow/feast-mlflow-project/feature_repo.



In [35]:
df.to_parquet('feature_repo/feature_repo/data/iris_stats.parquet')

In [36]:
with open("feature_repo/feature_repo/example_repo.py", "w") as my_frepo:
    my_frepo.write("""from datetime import timedelta

from feast import Entity, FeatureView, Field, FileSource, ValueType, FeatureService
from feast.types import Float32, Int64

iris_stats = FileSource(
    path="/feature_repo/feature_repo/data/iris_stats.parquet",
    timestamp_field="event_timestamp",
)

iris = Entity(name="iris", value_type=ValueType.INT64, join_keys=["iris_id"], description="iris id",)

# Our parquet files contain sample data that includes a driver_id column, timestamps and
# three feature column. Here we define a Feature View that will allow us to serve this
# data to our model online.
iris_stats_fv = FeatureView(
    name="iris_stats_view",
    entities=["iris"],  # reference entity by name
    ttl=timedelta(days=1),
    schema=[
        Field(name="sepal_length_cm", dtype=Float32),
        Field(name="sepal_width_cm", dtype=Float32),
        Field(name="petal_length_cm", dtype=Float32),
        Field(name="petal_width_cm", dtype=Float32),
        Field(name="target", dtype=Int64),
    ],
    online=True,
    source=iris_stats,
    tags={},
)

# iris_stats_fs = FeatureService(
#     name="iris_stats_fs",
#     features=iris_stats_fv
# )
""")

In [38]:
!cd feature_repo/feature_repo && feast apply

Traceback (most recent call last):
  File "/home/sbalawajder/projects/train/feast-mlflow/feast-mlflow-project/venv/bin/feast", line 8, in <module>
    sys.exit(cli())
  File "/home/sbalawajder/projects/train/feast-mlflow/feast-mlflow-project/venv/lib/python3.9/site-packages/click/core.py", line 1130, in __call__
    return self.main(*args, **kwargs)
  File "/home/sbalawajder/projects/train/feast-mlflow/feast-mlflow-project/venv/lib/python3.9/site-packages/click/core.py", line 1055, in main
    rv = self.invoke(ctx)
  File "/home/sbalawajder/projects/train/feast-mlflow/feast-mlflow-project/venv/lib/python3.9/site-packages/click/core.py", line 1657, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "/home/sbalawajder/projects/train/feast-mlflow/feast-mlflow-project/venv/lib/python3.9/site-packages/click/core.py", line 1404, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "/home/sbalawajder/projects/train/feast-mlflow/feast-mlflow-project/ven